## 关于BCEloss与CEloss的探究
对于BCEloss多用于二分类问题，但是很多多标签问题也使用到了BCEloss，一个样本有可能属于多个标签，这样每一个标签都是一个二分类，是一个伯努利分布并非多项式分布（softmax，CEloss）。注意二分类问题BCEloss结果应等同于CEloss但是两者代表的含义却不同（真正应用中结果也不可能相同，BCEloss经过linear层只需要输出一个通道，而如果使用CEloss的话需要将最后的输出应为两个通道，否则经过softmax输出都为1，通常二分类使用BCEloss）。但在最近的一篇对ResNet的优化论文中提到了用BCEloss代替CEloss做多分类问题（并非多标签），应该对标签做处理，若分n类则linear层应输出n个通道，标签也需要扩展到n维原先的标签值变成扩展后的正标记的下标。

In [1]:
import torch
from torch import nn

### 二分类示例：
$$
BCEloss(x_i,y_i) = -mean(y_ilog(x_i)+(1-y_i)log(1-x_i))
$$

In [54]:
# 随机生成五个标准正态分布的矩阵
test = torch.randn((5,1))
table = torch.randint(0,2,(5,1)).float()
test, table

(tensor([[-0.0564],
         [ 1.1831],
         [-0.2914],
         [ 0.7953],
         [-1.8517]]),
 tensor([[0.],
         [0.],
         [1.],
         [1.],
         [0.]]))

In [55]:
sig = nn.Sigmoid()(test)
sig

tensor([[0.4859],
        [0.7655],
        [0.4277],
        [0.6890],
        [0.1357]])

In [57]:
# bceloss
mun = ((-torch.log(sig)*table)+(-torch.log(1-sig)*(1-table))).mean() # 手写bceloss
celoss = nn.BCEWithLogitsLoss()(test, table) # torch api
celoss==mun
# 注意二分类问题如果使用CEloss的话需要将最后的输出应设为两列，否则经过softmax输出都为1

tensor(True)

### 二分类中的另一种情况，输出为2维的BCEloss与CEloss对比

In [149]:
test_bi = torch.randn((5,2))
table_bi = torch.zeros_like(test_bi)
table_bi[[0,1,2,3,4],table.long().view(-1,)] = 1
test_bi, table_bi, table
# table_bi和table分别对应bceloss与celoss的标签

(tensor([[-0.7511,  0.1949],
         [-3.2645,  1.0096],
         [ 1.1170, -1.1953],
         [ 2.4201, -1.0242],
         [-1.0678,  0.4846]]),
 tensor([[1., 0.],
         [1., 0.],
         [0., 1.],
         [0., 1.],
         [1., 0.]]),
 tensor([[0.],
         [0.],
         [1.],
         [1.],
         [0.]]))

In [150]:
# 2输出bceloss
sig = nn.Sigmoid()(test_bi)
bceloss = -(torch.log(sig)*table_bi+torch.log(1-sig)*(1-table_bi)).mean()
bce2 = nn.BCEWithLogitsLoss()(test_bi, table_bi)
# 2输出celoss
celoss = nn.CrossEntropyLoss()(test_bi, table.view(-1,).long())
bceloss, bce2, celoss

(tensor(1.5579), tensor(1.5579), tensor(2.6378))

可以看到这种情况下BCEloss与CEloss的结果依然不同，体现了celoss经过softmax的可加性（相加为一）

### 多分类示例：
生成包含五个样本三个类别的示例，分别用BCEloss和CEloss查看其区别

In [114]:
# 随机生成标准正态分布的的矩阵，五个样本三种分类
test_multi = torch.randn((5,3))
sub_table = torch.randint(0,3,(5,))
table_multi = torch.zeros_like(test_multi)
# 将对应的下标转换成正例
table_multi[[0,1,2,3,4],sub_table]=1
test_multi, sub_table, table_multi

(tensor([[ 1.3216, -1.2250, -1.8526],
         [ 0.3159, -0.4316, -1.5087],
         [ 0.4384, -1.7315, -1.9941],
         [-0.8894,  0.9585, -1.1600],
         [ 0.2089, -0.9754,  0.0882]]),
 tensor([1, 2, 2, 2, 1]),
 tensor([[0., 1., 0.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 1., 0.]]))

In [115]:
# 手写BCEloss用于对照
sig = nn.Sigmoid()(test_multi)
manual = ((-torch.log(sig)*table_multi)+(-torch.log(1-sig)*(1-table_multi))).mean() # 手写celoss
celoss = nn.BCEWithLogitsLoss()(test_multi, table_multi) # torch api
print("手写BCEloss与api计算方式相同") if '%.06f'%celoss.item()=='%.06f'%manual.item() else print("手写BCEloss与api计算方式不同")
print(celoss.item(),manual.item())
try:
    nn.BCEWithLogitsLoss()(test_multi, sub_table)
except Exception as e:
    print("BCEloss不能使用一个维度的标签应扩展到类别维度，报错信息为：")
    print(e)

手写BCEloss与api计算方式相同
1.0250824689865112 1.0250824689865112
BCEloss不能使用一个维度的标签应扩展到类别维度，报错信息为：
Target size (torch.Size([5])) must be the same as input size (torch.Size([5, 3]))


可以看到手写同api输出结果相同，中间过程由于精度的差异略有不同,下面测试多分类问题中BCEloss与CEloss的区别

In [117]:
soft_mutil = nn.Softmax(dim=1)(test_multi)
true_table = soft_mutil[[0,1,2,3,4],sub_table]
celoss_manual = -torch.log(true_table).mean()
celoss_api = nn.CrossEntropyLoss()(test_multi, sub_table)
print("手写CEloss与api计算方式相同") if celoss_manual==celoss_api else print("手写CEloss与api计算方式不同")
print(celoss_manual, celoss_api)

手写CEloss与api计算方式相同
tensor(2.3851) tensor(2.3851)


经过上面的实验，可以看到相同的标记和样本情况下，BCEloss与CEloss是完全不同的，BCEloss在多分类非多标签任务中通过改变标签的表示方式也是可以使用的，另外需要注意的是经过上述实验在制作数据集的时候，多分类问题BCEloss的标签是n维的（与特征维度相同的下标编码），而CEloss是一维的。二分类问题中BCEloss最后经过linear是一维的，标签是一维的，而CEloss经过linear层输出是二维的，标签是一维的。